## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 라이브러리`

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. 도구 호출 (Tool Calling)
- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음 

### 2-1. 랭체인 내장 도구
- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

In [3]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearchResults(max_results=2)

# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력
for result in search_results:
    print(result)  
    print("-" * 100)  

{'url': 'https://m.blog.naver.com/wineislikeacat/223096696241', 'content': '중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\n\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\n\n\u200b\n\n이에 맞는 레드와인으로는\n\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\n\n\u200b\n\n![Image 7](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTEw/MDAxNjgzNTQ1MDQ3NDI0.v4ZHSvlETicMdag0DaoWGFsrTMFezoiZD_41J0Htwhsg.mPiLhr9soLZMXEOb_GJatpnqVUUNFZkrcmQRn9omaewg.JPEG.wineislikeacat/clark-douglas-7WJV-SDSj6c-unsplash.jpg?type=w80_blur)\n\n안심 스테이크와 어울리는 와인\n\n> **안심 스테이크와 어울리는 와인 품종은? 산지오베제!**\n\n고기 본연의 맛을 즐기기 가장 좋은 부위로 꼽히는 안심은 [...] ![Image 6](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTQ4/MDAxNjgzNTQ0MjY4OTc2.FasX2MpcBaCDnf0qjWA_PcbLTIlySdPLi7FFnJlTboog.AQwrp1etohVQ3fiIHA0ga9lD45uGexCOZzB_1C12eGQg.JPEG.wineislikeacat/paul-einerhand-dwtt3noJG2k-unsplash.jpg?type=w800)\n\n등심 & 채끝 스테이크와 어울리는 와인\n\n> **등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!**\n\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\n\n육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\n\n채끝은 등뼈에서 허리

In [4]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search_results_json
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. Input '
 'should be a search query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}
----------------------------------------------------------------------------------------------------

`(2) 도구(tool) 호출하기`

In [5]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])

In [6]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 82, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run--1b229b0e-b2a3-41ec-b5bb-c5dd03ae4e05-0', usage_metadata={'input_tokens': 82, 'output_tokens': 12, 'total_tokens': 94})
----------------------------------------------------------------------------------------------------
'안녕하세요! 무엇을 도와드릴까요?'
----------------------------------------------------------------------------------------------------
[]
----------------------------------------------------------------------------------------------------


In [7]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XSCqNsNGECdgtGhikSsI6o34', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_v76UDSiFpfirYPZLDKMszgb8', 'function': {'arguments': '{"query": "steak wine pairing recommendations"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 91, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--06a3678c-8335-4e53-92e8-22cf2f3a6531-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_

In [8]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_XSCqNsNGECdgtGhikSsI6o34',
 'type': 'tool_call'}

`(3) 도구(tool) 실행하기`

In [9]:
### 방법 1: 직접 도구 호출 처리

# 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
# 'args'를 사용하여 도구를 호출하고 결과를 얻는다.

tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)
print(tool_output)

tavily_search_results_json 호출 결과:
----------------------------------------------------------------------------------------------------
[{'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80', 'content': '1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n------------------------------------------------\n\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n\n**추천 이유:**\n\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\n\n**어울리는 스테이크:**\n\n*   리브아이 스테이크\n*   뉴욕 스트립\n*   티본 스테이크\n\n**추천 와인:**\n\n*   나파 밸리 카베르네 소비뇽\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/im

In [10]:
### 방법 2: ToolMessage 객체 생성

# 이 방법은 도구 호출 결과를 사용하여 ToolMessage 객체를 생성한다.
# 도구 호출의 ID와 이름을 포함하여 더 구조화된 메시지를 만든다.

from langchain_core.messages import ToolMessage
tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

pprint(tool_message)

ToolMessage(content=[{'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80', 'content': '1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n------------------------------------------------\n\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n\n**추천 이유:**\n\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\n\n**어울리는 스테이크:**\n\n*   리브아이 스테이크\n*   뉴욕 스트립\n*   티본 스테이크\n\n**추천 와인:**\n\n*   나파 밸리 카베르네 소비뇽\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKprxC9mea7bnWY9Y65bk/img.webp)![Image 12](https

In [11]:
### 방법 3: 도구 직접 호출하여 바로 ToolMessage 객체 생성

# 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
# 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

tool_message = web_search.invoke(tool_call)

print(tool_message)


content='[{"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\\n------------------------------------------------\\n\\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKprxC9mea7bnWY9Y65bk/img.webp)![Image 12

In [12]:
pprint(tool_message.tool_call_id)

'call_XSCqNsNGECdgtGhikSsI6o34'


In [13]:
pprint(tool_message.name)

'tavily_search_results_json'


In [14]:
pprint(tool_message.content)

('[{"url": '
 '"https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", '
 '"content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 '
 '대표주자\\n------------------------------------------------\\n\\n![Image '
 '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
 '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 '
 '이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, '
 '고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   '
 '티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château '
 'Margaux) [...] ![Image '
 '10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image '
 '11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKpr

In [15]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': '스테이크와 어울리는 와인 추천'},
  'id': 'call_XSCqNsNGECdgtGhikSsI6o34',
  'type': 'tool_call'},
 {'name': 'tavily_search_results_json',
  'args': {'query': 'steak wine pairing recommendations'},
  'id': 'call_v76UDSiFpfirYPZLDKMszgb8',
  'type': 'tool_call'}]

In [16]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='[{"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\\n------------------------------------------------\\n\\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKprxC9mea7bnWY9Y65bk/img.we

`(4) ToolMessage를 LLM에 전달하여 답변을 생성하기`

In [17]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[web_search])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = web_search_chain.invoke("2020년 모엣샹동 샴페인의 가격은 얼마인가요?")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_ejfGFQCg6C6TaGcj1jKM3j9V', 'function': {'arguments': '{"query":"2020 Moët & Chandon champagne price"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 116, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--eac852cd-d8e7-46f1-bc90-1004c4399d83-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2020 Moët & Chandon champagne price'}, 'id': 'call_ejfGFQCg6C6TaGcj1jKM3j9V', 'type': 'tool_call'}] usage_metadata={'input_tokens': 116, 'output_tokens': 26, 'total_tokens': 142}
---------------

### 2-2. 사용자 정의 도구
- @tool decorator를 통해 사용자 정의 도구를 정의할 수 있음

`(1) 도구(tool) 정의하기`

In [18]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

# Tool 정의 
@tool
def search_web(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [19]:
# 도구 속성
print("자료형: ")
print(type(search_web))
print("-"*100)

print("name: ")
print(search_web.name)
print("-"*100)

print("description: ")
pprint(search_web.description)
print("-"*100)

print("schema: ")
pprint(search_web.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_web
----------------------------------------------------------------------------------------------------
description: 
('Searches the internet for information that does not exist in the database or '
 'for the latest information.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Searches the internet for information that does not exist in '
                'the database or for the latest information.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_web',
 'type': 'object'}
----------------------------------------------------------------------------------------------------


In [20]:
query = "스테이크와 어울리는 와인을 추천해주세요."
search_result = search_web.invoke(query)

print(search_result)

<Document href="https://m.blog.naver.com/wineislikeacat/223096696241"/>
중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면

씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.

​

이에 맞는 레드와인으로는

카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!

​

![Image 7](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTEw/MDAxNjgzNTQ1MDQ3NDI0.v4ZHSvlETicMdag0DaoWGFsrTMFezoiZD_41J0Htwhsg.mPiLhr9soLZMXEOb_GJatpnqVUUNFZkrcmQRn9omaewg.JPEG.wineislikeacat/clark-douglas-7WJV-SDSj6c-unsplash.jpg?type=w80_blur)

안심 스테이크와 어울리는 와인

> **안심 스테이크와 어울리는 와인 품종은? 산지오베제!**

고기 본연의 맛을 즐기기 가장 좋은 부위로 꼽히는 안심은 [...] ![Image 6](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTQ4/MDAxNjgzNTQ0MjY4OTc2.FasX2MpcBaCDnf0qjWA_PcbLTIlySdPLi7FFnJlTboog.AQwrp1etohVQ3fiIHA0ga9lD45uGexCOZzB_1C12eGQg.JPEG.wineislikeacat/paul-einerhand-dwtt3noJG2k-unsplash.jpg?type=w800)

등심 & 채끝 스테이크와 어울리는 와인

> **등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!**

등심은 소의 등뼈를 둘러싸고 있는 부위입니다.

육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.

채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,

마블링 러버들의 선택을 쉽사리 받

In [21]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[search_web])

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XbTGQUqFpny9pRMtSOrtdt9u', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_OCodgDzXaeX8Ekoq56zOk2fK', 'function': {'arguments': '{"query": "best wine pairings for steak"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 67, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3d9d71e9-1e7b-458f-b330-73e1d142153e-0', tool_calls=[{'name': 'search_web', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_XbTGQUqFpny9pRMtSOrtdt9u', 'type': 'tool_call'}, {'nam

`(2) LLM 도구 호출 성능 비교하기`

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

# 기본 LLM
llm_gemini_flash = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
llm_gemini_pro = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
llm_groq = ChatGroq(model="llama3-70b-8192", temperature=0)

# LLM에 도구 바인딩하여 추가 
tools=[search_web]

gemini_flash_with_tools = llm_gemini_flash.bind_tools(tools)
gemini_pro_with_tools = llm_gemini_pro.bind_tools(tools)
groq_llama3_with_tools = llm_groq.bind_tools(tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


- gemini-1.5-flash

In [23]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = gemini_flash_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='스테이크와 어울리는 와인은 스테이크의 종류와 조리 방법에 따라 다릅니다.  좀 더 구체적으로 어떤 종류의 스테이크인지, 어떻게 조리되었는지 알려주시면 더욱 정확한 추천을 드릴 수 있습니다.\n\n하지만 일반적으로 다음과 같은 와인들이 스테이크와 잘 어울립니다.\n\n* **레드 와인:**  풍부한 탄닌과 강한 바디감을 가진 와인이 스테이크의 풍미와 잘 어울립니다.  \n    * **카베르네 소비뇽 (Cabernet Sauvignon):**  풀바디, 탄닌이 풍부하고 블랙커런트, 블랙체리, 시가향 등의 풍미를 지니고 있어, 특히 잘 구워진 스테이크와 잘 어울립니다.\n    * **말벡 (Malbec):**  부드러운 탄닌과 다크 과일 향이 특징이며,  중간 정도의 바디감을 가지고 있어 다양한 스테이크와 잘 어울립니다.\n    * **메를로 (Merlot):**  부드럽고 매끄러운 질감과 블랙베리, 체리 향이 특징이며,  카베르네 소비뇽보다 탄닌이 적어 부드러운 스테이크와 잘 어울립니다.\n    * **쉬라즈/시라 (Shiraz/Syrah):**  강한 바디감과 스파이시한 향이 특징이며,  풍성한 풍미의 스테이크와 잘 어울립니다.\n\n\n* **화이트 와인:**  스테이크의 종류에 따라 화이트 와인도 잘 어울릴 수 있습니다.  \n    * **샤르도네 (Chardonnay):**  오크 숙성된 샤르도네는 버터리하고 풍부한 풍미를 가지고 있어, 크림 소스를 곁들인 스테이크와 잘 어울립니다.\n\n\n어떤 종류의 스테이크인지, 어떻게 조리되었는지 (예: 미디엄 레어, 웰던 등) 알려주시면 더욱 구체적인 와인 추천을 해드리겠습니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f1

- gemini-1.5-pro

In [24]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = gemini_pro_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='스테이크와 잘 어울리는 와인을 추천해 드리겠습니다. 스테이크의 종류와 굽기에 따라 와인의 종류도 달라집니다. 어떤 종류의 스테이크를 드시는지, 그리고 어떤 굽기로 드시는지 알려주시면 더욱 자세한 추천을 해 드릴 수 있습니다.\n\n일반적으로 다음과 같은 와인들이 스테이크와 잘 어울립니다.\n\n* **까베르네 소비뇽 (Cabernet Sauvignon):** 풀바디 와인으로 탄닌이 풍부하고 블랙커런트, 블랙베리, 삼나무 향이 납니다. 등심, 안심, 티본 스테이크 등 대부분의 스테이크와 잘 어울립니다. 특히 미디엄 레어나 미디엄으로 구운 스테이크와 좋은 궁합을 보입니다.\n\n* **메를로 (Merlot):** 부드러운 탄닌과 자두, 체리, 초콜릿 향이 나는 와인입니다. 까베르네 소비뇽보다 부드러워 부드러운 육질의 스테이크와 잘 어울립니다. 안심이나 등심 스테이크와 함께 드시면 좋습니다.\n\n* **피노 누아 (Pinot Noir):** 가벼운 바디감과 섬세한 탄닌을 가진 와인으로 체리, 라즈베리, 버섯 향이 납니다. 섬세한 맛의 스테이크와 잘 어울리며, 특히 안심 스테이크와 좋은 궁합을 보입니다.\n\n* **진판델 (Zinfandel):** 풀바디 와인으로 블랙베리, 라즈베리, 후추 향이 납니다. 진한 풍미의 스테이크와 잘 어울리며, 립아이 스테이크와 함께 드시면 좋습니다.\n\n* **쉬라즈/시라 (Syrah/Shiraz):** 풀바디 와인으로 블랙베리, 블루베리, 스파이스 향이 납니다. 탄닌이 풍부하고 진한 풍미를 가지고 있어 립아이, 티본 스테이크와 잘 어울립니다.\n\n\n스테이크의 종류와 굽기를 알려주시면 더욱 구체적인 와인을 추천해 드리겠습니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [

- llama3-70b-8192

In [25]:
# 도구 호출이 필요한 LLM 호출을 수행

# 도구를 꼭 쓰라고 강제한 게 아니라면, LLM마다 도구를 쓸지 말지 자체적으로 판단함.

# Gemini Pro는 보통 자체 지식이 충분해서 답을 직접 하려 하고,
# 반면 Groq LLaMA3는 "도구를 쓸래요" 하고 tool_call을 만들어냄.


query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = groq_llama3_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)  

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pfvc', 'function': {'arguments': '{"query":"wine pairing for steak"}', 'name': 'search_web'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 900, 'total_tokens': 946, 'completion_time': 0.208280798, 'prompt_time': 0.028995937, 'queue_time': 0.220673853, 'total_time': 0.237276735}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--24305a10-ec63-42bc-bc18-c1adbab98c9d-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'wine pairing for steak'}, 'id': 'call_pfvc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 900, 'output_tokens': 46, 'total_tokens': 946})
----------------------------------------------------------------------------------------------------
''
----------------------------------------------------------------------------------------------------
[{'args': {'query

### 2-3. Runnable 객체를 도구(tool) 변환
- 문자열이나 dict 입력을 받는 Runnable을 도구로 변환
- as_tool 메서드를 사용

`(1) Document Loader`

In [26]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    
    # 이 부분이 바로 LangChain의 WikipediaLoader를 통해 Wikipedia API를 호출하는 부분이야.
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)

C:\Users\USER\AppData\Local\Temp\ipykernel_2484\608300787.py:26: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_search = runnable.as_tool(


In [27]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_search
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a specified number of documents. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'k': {'default': 2,
                      'description': 'The number of documents to return '
                                     '(default is 2)',
                      'title': 'K',
                      'type': 'integer'},
                'q

In [28]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 도구만 실행함
# LLM은 관여하지 않음
# 순수하게 "파스타의 유래"라는 query를 wiki 함수가 처리

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

page_content='이탈리아 요리(Italia 料理, 이탈리아어: cucina italiana 쿠치나 이탈리아나[*])는 남유럽에 있는 이탈리아의 요리이다. 기원전 4세기부터 다양한 사회·정치 변화와 함께 발전해 왔다. 특히 유럽인들이 세계 각지로 진출한 이후 신대륙에서 감자, 토마토, 단고추, 옥수수 등의 다양한 식재료가 유입되면서 이탈리아 요리는 큰 변화를 겪었다. 이런 재료는 18세기 이전까지는 보급되지 않았다.
이탈리아 요리는 각 지역마다 고유한 특색이 있고 다양하지만 크게 북부와 남부로 구분한다. 다른 나라와 국경을 맞대던 북부는 산업화하여 호황을 누리면서 농업이 발달한 덕분에 쌀이나 유제품이 사용되는 요리가 많지만 경제적으로 침체됐던 남부는 지역 특산물인 올리브와 토마토, 모짜렐라 치즈나 해산물을 활용한 요리가 많다.
간결함으로 귀결되는 특징이 있어 많은 경우 4개~8개 정도의 재료로 조리되며 정교한 준비 과정보다는 재료의 질에 따라 맛이 좌우된다. 치즈와 포도주는 이탈리아 요리의 핵심을 차지하며 여러 종류가 존재하는데 식재료와 치즈의 차이는 파스타의 종류와 요리에 기본이 되는 수프, 소스의 차이를 뜻한다.
2013년 CNN은 세계 최고의 요리로 이탈리아 요리를 선정했다.


== 역사 ==
대략 기원전 4세기부터 시작되었다고 여겨진다. 여러 세기 동안 주변 지역과 정복자, 유명한 요리사, 정치 변동 및 신대륙의 발견 등 매우 다양한 요소가 이탈리아 요리의 형성에 영향을 끼쳤다.


=== 고대 ===
이탈리아 요리에 대해 최초의 저술을 남긴 사람은 그리스계 시칠리아 사람인 아케스트라투스이다. 기원전 4세기에 지금의 시라쿠사 지역에 살던 그는 “계절에 맞는 최상의 재료”를 사용하는 것에 대한 시를 남겼다. 그는 또한 향신료나 허브, 양념 따위를 사용한 음식과 그 중요성, 생선 요리 등에 대해 서술했다. 특정 양념이나 향신료가 지나쳐서는 안 된다는 것 또한 집필 내용에 포함됐지만 470가지의 요리법을 담은 책인 《데 레 코퀴나리아》(De re c

In [29]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)

# 여기서 실질적으로 llm(GPT 4o, 최상단에서 설정) 과 툴이 바인딩된다. 
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TLVFaoefDxgBxRw1I7MZJxOP', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_wa6VmhTuCiIbjdXeXhqOKnWV', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 174, 'total_tokens': 230, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--83a682c4-f4bc-4a13-b3b2-d60f4356345b-0', tool_calls=[{'name': 'search_web', 'args': {'query': '서울 강남 파스타 맛집 추천'}, 'id': 'call_TLVFaoefDxgBxRw1I7MZJxOP', 'type': 'tool_call'}, {'name': 'wiki_search', 'a

`(2) LCEL 체인`
- 위키피디아 문서를 검색하고 내용을 요약하는 체인

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

('이탈리아 요리(쿠치나 이탈리아나)는 기원전 4세기부터 발전해온 남유럽 이탈리아의 전통 요리로, 지역마다 고유한 특색이 있다. 북부는 쌀과 '
 '유제품을, 남부는 올리브와 토마토, 해산물을 주로 사용한다. 이탈리아 요리는 간결함을 특징으로 하며, 재료의 질이 맛에 큰 영향을 '
 '미친다. CNN은 2013년에 이탈리아 요리를 세계 최고의 요리로 선정했다. 요리의 역사적 발전은 고대 그리스와 로마, 중세 아랍 문화의 '
 '영향을 받았으며, 다양한 요리책이 출판되면서 정교한 요리법이 소개되었다. 오르조는 이탈리아의 파스타로, 보리에서 유래했으며, 현재는 '
 '박력분으로 만들어진다.')


In [31]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [32]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VgSckCjjf1v3ndCv7k1z4U8q', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_ykQds8mNcXRSts1FQBFx1cie', 'function': {'arguments': '{"query": "파스타"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 150, 'total_tokens': 203, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e48b2506-08ea-463d-8fef-34e7cacd6eb7-0', tool_calls=[{'name': 'search_web', 'args': {'query': '서울 강남 파스타 맛집 추천'}, 'id': 'call_VgSckCjjf1v3ndCv7k1z4U8q', 'type': 'tool_call'}, {'name': 'wiki_summary', 'arg

In [33]:
ai_msg.tool_calls[1]

{'name': 'wiki_summary',
 'args': {'query': '파스타'},
 'id': 'call_ykQds8mNcXRSts1FQBFx1cie',
 'type': 'tool_call'}

In [34]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])

print(tool_message)
print("-" * 100)
pprint(tool_message.content)

content='파스타는 이탈리아의 주요 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 국민 음식으로 여겨지며, 역사적으로 그리스와 아랍 문화의 영향을 받았다. 파스타는 건파스타와 생파스타로 나뉘며, 다양한 형태와 종류가 존재한다. 요리 방법으로는 삶거나 오븐에 구워내는 방식이 있다. 투움바 파스타는 미국의 아웃백 스테이크하우스에서 유래된 요리로, 매콤한 크림 소스를 사용하며 한국에서 인기를 끌고 있다.' name='wiki_summary' tool_call_id='call_ykQds8mNcXRSts1FQBFx1cie'
----------------------------------------------------------------------------------------------------
('파스타는 이탈리아의 주요 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 국민 '
 '음식으로 여겨지며, 역사적으로 그리스와 아랍 문화의 영향을 받았다. 파스타는 건파스타와 생파스타로 나뉘며, 다양한 형태와 종류가 '
 '존재한다. 요리 방법으로는 삶거나 오븐에 구워내는 방식이 있다. 투움바 파스타는 미국의 아웃백 스테이크하우스에서 유래된 요리로, 매콤한 '
 '크림 소스를 사용하며 한국에서 인기를 끌고 있다.')


In [35]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_x1aHj1JaK4Ce4hhVQxqUyRLY', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--c27e94c9-b5c1-48db-87b0-6349d9888a14-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_x1aHj1JaK4Ce4hhVQxqUyRLY', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139}
---------------------------------------------------------------------------------------------------

### 2-4. 벡터저장소 검색기
- @tool decorator 사용

`(1) 문서 로드 및 인덱싱`

In [36]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [37]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...

메뉴 번호: 2
메뉴 이름: 트러플 리조또
내용:
2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


In [38]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Chroma 인덱스 생성
menu_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 2개
메뉴 번호: 1
메뉴 이름: 시그니처 스테이크

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크



- 와인 메뉴에 대해서도 같은 작업을 처리

In [39]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...
검색 결과: 2개
메뉴 번호: 6
메뉴 이름: 바롤로 몬프리바토 2017

메뉴 번호: 6
메뉴 이름: 바롤로 몬프리바토 2017



`(2) 도구(tool) 정의하기`

In [40]:
# 벡터 저장소 로드
menu_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

@tool
def search_menu(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(search_menu))
print("-"*100)

print("name: ")
print(search_menu.name)
print("-"*100)

print("description: ")
pprint(search_menu.description)
print("-"*100)

print("schema: ")
pprint(search_menu.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_menu
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant menu '
                'information from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_menu',
 'type': 'object'}
----------------

In [41]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# 벡터 저장소 로드
wine_db = Chroma(
   embedding_function=embeddings_model,   
   collection_name="restaurant_wine",
   persist_directory="./chroma_db",
)

@tool
def search_wine(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(search_wine))
print("-"*100)

print("name: ")
print(search_wine.name)
print("-"*100)

print("description: ")
pprint(search_wine.description)
print("-"*100)

print("schema: ")
pprint(search_wine.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_wine
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant wine information from the '
 'encrypted database.\n'
 'Use this tool only for wine-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant wine '
                'information from the encrypted database.\n'
                'Use this tool only for wine-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_wine',
 'type': 'object'}
----------------

In [42]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[search_menu, search_wine])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YcuRa54gbQA5Mp4onozfCDSA', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'search_menu'}, 'type': 'function'}, {'id': 'call_9lmH1w8sf9wj7qnvNUDTfxbm', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'search_wine'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e2eb841d-5159-4353-8022-76e88ad4285e-0', tool_calls=[{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_YcuRa54gbQA5Mp4onozfCDSA', 'type': 'tool_call'}, {'name': 'search_wine', 'args': {'query

`(3) 여러 개의 도구(tool) 호출하기`

In [43]:
tools = [search_web, wiki_summary, search_wine, search_menu]
for tool in tools:
    print(tool.name) 

search_web
wiki_summary
search_wine
search_menu


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")

# 응답 출력 
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_cXPwd8ll3GVMWLi8qRdc7YHd', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_wine: 
{'name': 'search_wine', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_xv1hfdrShDrZqL7gAn1PF3Fe', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,00

In [45]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '파스타'}, 'id': 'call_XZtJ1OjGoNoBsSiRv42YXgOe', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타'}, 'id': 'call_UcrWkQB4dvBF32CgxsGolAvY', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '해산물 파스타', 'menu_number': 6, 'source': './data/restaurant_menu.txt'}, page_content='6. 해산물 파스타\\n   • 가격: ₩24,000\\n   • 주요 식재료: 링귀네 파스타, 새우, 홍합, 오징어, 토마토 소스\\n   • 설명: 알 덴테로 삶은 링귀네 파스타에 신선한 해산물을 듬뿍 올린 메뉴입니다. 토마토 소스의 산미와 해산물의 감칠맛이 조화를 이루며, 마늘과 올리브 오일로 풍미를 더했습니다. 파슬리를 뿌려 향긋한 맛을 더합니다.'), Document(metadata={'menu_name': '해산물 파스타', 'menu_number': 6, 'source': './data/restaurant_menu.txt'}, page_content='6. 해산물 파스타\\n   • 가격: ₩24,000\\n   • 주요 식재료: 링귀네 파스타, 새우, 홍합, 오징어, 토마토 소스\\n 

## 3. Few-shot 프롬프팅 
- 각 도구의 용도를 구분하여 few-shot 예제로 제시

### 3-1. Few-shot 도구 호출

In [46]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_menu", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. 주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_wine", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. 2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, 주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. 특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오, 그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_menu', 'args': {'query': '스테이크'}, 'id': 'call_kOYbKO6ObHFaykfoy1zPLBah', 'type': 'tool_call'}
{'name': 'search_wine', 'args': {'query': '스테이크에 어울리는 와인'}, 'id': 'call_l0yRGeiBv1U0eNbZeNf1dGvx', 'type': 'tool_call'}


In [47]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_9JaZSRmJBmoa6bSGjOm4LWTi', 'type': 'tool_call'}
{'name': 'search_web', 'args': {'query': '서울 강남 파스타 맛집 추천'}, 'id': 'call_usqTDP82tdnwrf9GeH903D8w', 'type': 'tool_call'}


### 3-2. 답변 생성 체인 

In [48]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '스테이크'}, 'id': 'call_Woy59hRJkTZKSDbSZS57oZdX', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_wine: 
{'name': 'search_wine', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_FMDxDtp2S8gZHCfDJ6IF3Xx7', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n 

In [49]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_Xwft5ASySGyZgmIxoJoeSvDQ', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_menu: 
{'name': 'search_menu', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_2ZCyidv7ze0U1kXBxe9QzAey', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content='이탈리아 요리(쿠치나 이탈리아나)는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 고유한 특색이 있으며 북부는 쌀과 유제품, 남부는 올리브와 해산물을 주로 사용한다. 이 요리는 간결함을 특징으로 하며, 재료의 질이 맛에 큰 영향을 미친다. CNN은 2013년에 이탈리아 요리를 세계 최고의 요리로 선정했다. 역사적으로, 이탈리아 요리는 고대 그리스와 로마의 영향을 받았고, 중세에는 아랍 문화와의 접촉으로 변화가 있었다. 근대에는 궁중 요리 문화가 발전하며 다양한 요리책이 출판되었다. 오르조는 이탈리아의 파스타로, 보리에서 유래했으며, 현재는 박력분으로 만들어진다.', name='wiki_summary', tool_call_id='call_Xwft5ASySGyZgmIxoJoeSvDQ'), ToolMessage(content="[Document(metadata={'menu_name': '해산물 파스타', 'menu_number': 6, 'source': './data/restaur

## 4. LangChain Agent 사용
- 유의사항: 프롬프트에 "agent_scratchpad",  "input" 변수를 포함

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [51]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [search_web, wiki_summary, search_wine, search_menu]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [52]:
# AgentExecutor 실행

query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `search_menu` with `{'query': '시그니처 스테이크'}`


[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.')]
Invoking: `search_wine` with `{'query': '스테이크와 어울리는 와인'}`


[Document(metadata={'menu_name': '바롤로 몬프리바토 2017', 'menu_number': 6, 'source': './data/restaurant_wine.txt'}, page_content='6. 바롤로 몬프리바토 2017\n   • 

In [53]:
pprint(agent_response)

{'input': '시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.',
 'output': '### 시그니처 스테이크 정보\n'
           '- **가격**: ₩35,000\n'
           '- **주요 재료**: 생등심, 로즈마리 간장, 그릴드 아스파라거스\n'
           '- **특징**: 이 스테이크는 등급이 높은 생등심을 사용하여 고소하고 부드러운 맛이 특징입니다. 로즈마리 간장을 '
           '곁들여 풍미를 더욱 강조하며, 그릴에 구운 아스파라거스를 함께 제공하여 신선함을 더합니다.\n'
           '\n'
           '### 스테이크와 어울리는 와인 추천\n'
           '- **와인 이름**: 바흘레 라페리카르 2017\n'
           '- **가격**: ₩280,000\n'
           '- **주요 특징**: 이 와인은 깊고 복합적인 향과 맛을 지니고 있으며, 고기 요리와의 궁합이 좋습니다. 특히 탄닌이 '
           '부드럽고, 과일 향과 스파이스가 어우러져 스테이크의 풍미를 더욱 끌어올려줍니다.\n'
           '\n'
           '스테이크와 잘 어울리는 이 와인은 고급스러운 식사 경험을 제공할 것입니다. 함께 즐기시면 좋을 것 같아요! 😊'}


## 5. Gradio 활용

In [54]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [55]:
# 데모 종료
demo.close()

Closing server running on port: 7860
